# Penyelesaian Soal ETL Green Finance Assessment

In [2]:
!pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load datasets
env_df = pd.read_excel("C:/Users/046ITAVT/Documents/046ITAVT/Training/A9046DB2025/202506/Ujian/Environmental_Dataset.xlsx")
fin_df = pd.read_excel("C:/Users/046ITAVT/Documents/046ITAVT/Training/A9046DB2025/202506/Ujian/Financial_Dataset.xlsx")
econ_df = pd.read_excel("C:/Users/046ITAVT/Documents/046ITAVT/Training/A9046DB2025/202506/Ujian/Economic_Dataset.xlsx")
social_df = pd.read_excel("C:/Users/046ITAVT/Documents/046ITAVT/Training/A9046DB2025/202506/Ujian/Social_Dataset.xlsx")
geo_df = pd.read_excel("C:/Users/046ITAVT/Documents/046ITAVT/Training/A9046DB2025/202506/Ujian/Geospatial_Dataset.xlsx")


## Soal 1: If-Else dan Aritmatika

In [7]:
merged_1 = pd.merge(env_df, fin_df, on="Project_ID")
plts = merged_1[merged_1["Project_ID"].str.startswith("PLTS")]

for _, row in plts.iterrows():
    try:
        ratio = row["CO2_Reduction"] / (row["Investment_Cost"] * 1_000)
        category = "High" if ratio >= 0.5 else "Low"
        print(f"{row['Project_ID']}: {ratio:.2f} ({category})")
    except ZeroDivisionError:
        print(f"{row['Project_ID']}: Cannot compute")

PLTS-NTT-001: 0.50 (High)
PLTS-JATIM-001: 0.45 (Low)
PLTS-SULS-001: 0.48 (Low)
PLTS-NTB-001: 0.44 (Low)
PLTS-JABW-001: 0.43 (Low)


## Soal 2: For Loop dan List

In [8]:
pltm_list = [row["CO2_Reduction"] for _, row in env_df.iterrows() if row["Project_ID"].startswith("PLTM")]
if pltm_list:
    avg = sum(pltm_list) / len(pltm_list)
    print(f"Average CO2 Reduction for PLTM Projects: {avg:.0f} tons CO2e")
else:
    print("No PLTM projects found.")

Average CO2 Reduction for PLTM Projects: 34600 tons CO2e


## Soal 3: While Loop dan Input (opsional)

In [9]:
social_dict = social_df.set_index("Project_ID")[["Land_Status", "Tingkat_Konflik"]].to_dict(orient="index")

while True:
    pid = input("Enter Project_ID (or 'DONE' to finish): ").strip()
    if pid.upper() == "DONE":
        break
    elif pid in social_dict:
        land = social_dict[pid]["Land_Status"]
        konflik = social_dict[pid]["Tingkat_Konflik"]
        print(f"{pid} - Land Status: {land}, Tingkat Konflik: {konflik}")
    else:
        print("Project not found")

Enter Project_ID (or 'DONE' to finish):  DONE


## Soal 4: Dictionary Filtering

In [10]:
merged_4 = pd.merge(econ_df, social_df, on="Project_ID")
project_info = {
    row["Project_ID"]: (row["Daya_Tarik_Investasi"], row["Tingkat_Konflik"])
    for _, row in merged_4.iterrows()
}

print("Projects with High Investment Attractiveness and Low Conflict:")
for pid, (attract, conflict) in project_info.items():
    if attract == "High" and conflict == "Low":
        print(pid)

Projects with High Investment Attractiveness and Low Conflict:


## Soal 5: Function

In [11]:
merged_5 = pd.merge(geo_df, fin_df, on="Project_ID")

def calculate_total_investment(project_ids, data):
    total = 0
    for pid in project_ids:
        row = data[data["Project_ID"] == pid]
        if not row.empty and row.iloc[0]["Efisiensi_Lokasi"] == "High":
            total += row.iloc[0]["Investment_Cost"]
    return total

ids_5 = merged_5["Project_ID"].tolist()
total = calculate_total_investment(ids_5, merged_5)
print(f"Total Investment for High-Efficiency Locations: {total:.2f} billion Rp")


Total Investment for High-Efficiency Locations: 0.00 billion Rp


## Soal 6: Fungsi Error Handling

In [13]:
def compute_co2_efficiency(co2_reduction, investment_cost):
    try:
        ratio = co2_reduction / (investment_cost * 1_000)
        return round(ratio, 2)
    except ZeroDivisionError:
        return "Cannot compute"

for pid in ["PLTS-NTT-001", "PLTS-JATIM-001", "PLTM-PAPU-001"]:
    row = merged_1[merged_1["Project_ID"] == pid]
    if not row.empty:
        co2 = row.iloc[0]["CO2_Reduction"]
        inv = row.iloc[0]["Investment_Cost"]
        result = compute_co2_efficiency(co2, inv)
        print(f"{pid}: {result}")
    else:
        print(f"{pid}: Data not found")


PLTS-NTT-001: 0.5
PLTS-JATIM-001: 0.45
PLTM-PAPU-001: 0.4


## Soal 7: Try-Except dalam Loop

In [14]:
energy_dict = env_df.set_index("Project_ID")["Energy_Output"].to_dict()
total_energy = 0
count = 0
for pid in ["PLTS-NTT-001", "PLTM-SULUT-002", "PLTA-JABAR-999", "PLTM-PAPU-001"]:
    try:
        energy = energy_dict[pid]
        total_energy += energy
        count += 1
    except KeyError:
        print(f"{pid}: Energy Output not found")
if count:
    avg_energy = total_energy / count
    print(f"Average Energy Output: {avg_energy:.0f} kWh")
else:
    print("No valid energy data found.")


PLTM-SULUT-002: Energy Output not found
PLTA-JABAR-999: Energy Output not found
Average Energy Output: 20000 kWh


## Bonus: Decision Tree

In [15]:
merged_all = econ_df.merge(env_df, on="Project_ID").merge(fin_df, on="Project_ID")
X = merged_all[["GDP_Growth", "CO2_Reduction", "Investment_Cost"]]
y = merged_all["Daya_Tarik_Investasi"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.2f}")
new_project = pd.DataFrame({"GDP_Growth": [5.0], "CO2_Reduction": [70000], "Investment_Cost": [150]})
print("Prediction for new project:", model.predict(new_project)[0])


Model Accuracy: 0.50
Prediction for new project: Medium: 💵💵💵
